## PAIR PROGRAMMING LIMPIEZA IV

### Valores Nulos
---

In [39]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sidetable

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de Limpieza anterior, el de Limpieza III, donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers.
​

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I y II ya dejamos algunas columnas limpitas:


- age ✔️ la limpiamos en el pair de Limpieza II
- species ✔️ la limpiamos en el pair de Limpieza II
- country ✔️ la limpiamos en el pair de Limpieza II
- fatal ✔️ la limpiamos en el pair de Pandas V
- year ✔️ es una columna de tipo float deberíamos convertirla a integer
- sex ✔️ la limpiamos en el pair de Pandas V
- fecha ✔️ la limpiamos en el pair de Pandas V

Es el momento de ponernos a trabajar con los valores nulos 💪🏽.

A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos, nosotras os planteamos los ejercicios pero sentiros libres de usar el método que más se adapte a vuestras necesidades. Manos a la obra!

In [40]:
df = pd.read_pickle("../files/attacks10.pkl")
df.sample(5)

,case_number,year,mes,sexo,edades,country,type,activity,fatal,cat_species
632,2007.09.03.b,2007,Sep,F,12.0,usa,Unprovoked,Jumping,N,Unspecified
161,ND.0009,1997,NaN,M,NaN,australia,Unprovoked,Fishing,Y,Unspecified
1068,1994.11.13,1994,Nov,F,34.0,usa,Unprovoked,"Surfing, paddling seawards",N,Unspecified
421,2012.09.24,2012,Sep,M,22.0,usa,Unprovoked,Surfing,N,Unspecified
100,ND.0081,1997,NaN,M,NaN,usa,Invalid,Diving,NaN,Unspecified


**EJERCICIO 1**

Lo primero que tenemos que evaluar es en que columnas tenemos nulos y que cantidad tenemos en cada una. ¿Hay alguna columna con una gran cantidad de nulos? En caso de que sea así deberemos eliminarla.

In [41]:
# Comprobamos la cantidad total y porcentaje de valores nulos del DataFrame con el método stb.missing
df.stb.missing().round(3)

,missing,total,percent
mes,170,1672,10.167
edades,162,1672,9.689
cat_species,126,1672,7.536
fatal,97,1672,5.801
activity,31,1672,1.854
sexo,14,1672,0.837
country,10,1672,0.598
case_number,0,1672,0.000
year,0,1672,0.000
type,0,1672,0.000


    Vemos que las columnas que tienen nulos no superan el 10% del total de los datos,  por lo que consideramos mas conveniente reemplazar los valores nulos por una nueva categoria para seguir teniendo el registro de esos ataques en vez de eliminarlos.

In [42]:
# Almacenamos en un diccionario los índices de las filas que tienen valores nulos iterando por cada columna
indices = [] 
for i in df.columns:
    ind = list(df[df[i].isnull()].index)
    indices.extend(ind)
    indices

In [43]:
# Convertimos el diccionario en set para eliminar los duplicados
indices = list(set(indices))
#indices

In [44]:
# Creamos un DataFrame con las filas que contienen valores nulos
df_nulos = df.iloc[indices]
df_nulos

,case_number,year,mes,sexo,edades,country,type,activity,fatal,cat_species
0,1800.00.00,1997,NaN,F,NaN,seychelles,Unprovoked,a corsair's boat was overturned,Y,NaN
1,1797.05.28.R,1997,May,NaN,NaN,NaN,Unprovoked,Dropped overboard,Y,NaN
2,1792.09.12,1997,Sep,M,NaN,england,Provoked,Fishing,Y,NaN
3,1791.00.00,1997,NaN,F,NaN,australia,Unprovoked,NaN,Y,NaN
4,1788.05.10,1997,May,NaN,NaN,australia,Boat,Fishing,N,NaN
...,...,...,...,...,...,...,...,...,...,...
494,2011.01.20,2011,Jan,F,7.0,australia,Invalid,Swimming,NaN,Unspecified
1521,1959.10.00,1959,NaN,M,36.0,fiji,Unprovoked,Spearfishing,N,Tiger
1523,1959.08.20,1959,Aug,M,10.0,philippines,Invalid,The 240-ton motor vessel Pilar II with 100 peo...,NaN,Unspecified
1020,1997.02.28.a,1997,Feb,M,NaN,australia,Unprovoked,Scuba diving,N,Tiger


---

**EJERCICIO 2**

Es el momento de eliminar los nulos:

- Reemplazad los valores nulos de la columna age por la media de la edad, redondeada a dos decimales.


In [45]:
# Utilizamos el método .fillna
df['edades'].fillna(df['edades'].mean().round(2), inplace=True)

In [46]:
df['edades'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'edades'

0

- En relación a la columna de country al tratarse de una columna de tipo categórica, reemplazad los valores nulos por una nueva categória que se llame Unknown.


In [47]:
# Utilizamos el método. replace
df['country'].replace(np.nan, 'Unknow', inplace=True)

In [48]:
df['country'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'country'

0

- Reemplazad los valores nulos de la columna fatal por Unknown.

In [49]:
# Utilizamos el método .replace
df['fatal'].replace(np.nan, 'Unknow', inplace=True)

In [50]:
df['fatal'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'fatal'

0

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda).


In [51]:
# Utilizamos el método .replace
df['type'].replace(np.nan, df.type.mode()[0], inplace=True)

In [52]:
df['type'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'type'

0

- Reemplazad los valores nulos de la columna fecha por Unknown.


In [53]:
# Utilizamos el método .replace
df['mes'].replace(np.nan, 'Unknow', inplace= True)

In [54]:
df['mes'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'mes'

0

In [55]:
df.stb.missing() # Comprobamos con el método stb.missing que no hay valores nulos en las cinco columnas modificadas en el ejercicio anterior

,missing,total,percent
cat_species,126,1672,7.535885
activity,31,1672,1.854067
sexo,14,1672,0.837321
case_number,0,1672,0.000000
year,0,1672,0.000000
mes,0,1672,0.000000
edades,0,1672,0.000000
country,0,1672,0.000000
type,0,1672,0.000000
fatal,0,1672,0.000000


---

**EJERCICIO 3**

Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair.


In [56]:
df.to_pickle('../files/attacks11.pkl')